# Producto CMIP del GOES-R

ABI Cloud and Moisture Imagery brightness temperature at top of atmosphere

El producto CMIP o MCMIP, contiene la información de reflectancia o temepratura de brillo, segun la región a la que corresponda la banda, estos son los insumos principales para las técnicas interpretación. En estos ejercicios trabajremos con los productos **MCMIPC**
por conveniencia, ya que estos consisten en un solo archivo con las 16 bandas del sensor ABI, todas remuestreadas a una resolución de 2 kilometros aproximadamente y la región de muestreo **CONUS**.

En este caso la convención empelada para codificar los nombres de las bandas es **CMI_C** y se utilizan los números del **01** al **16** para indicar la banda o canal a que corresponden. Estas no contienen ningun tipo de corrección atmosférica, aparte de la corrección por ángulo solar para las reflectancias. 

Estos archivos tambien incluyen una serie de variables auxilares para determinar la calidad de los datos, empezando por las variables DQF que consisten en una serie de banderas o etiquetas que te indican la calidad individual de cada uno de los pixels en la escena para la banda en cuestion. En este caso, son enteros del 0 al 4 que indican la calidad del pixel.

    0. good_pixel_qf
    1. conditionally_usable_pixel_qf
    2. out_of_range_pixel_qf
    3. no_value_pixel_qf
    4. focal_plane_temperature_threshold_exceeded_qf


In [ ]:
import xarray as xr
import holoviews as hv
import numpy as np

hv.extension('bokeh')

a_goes = 'datasets/OR_ABI-L2-MCMIPC-M6_G16_s20232412221172_e20232412223556_c20232412224050.nc'

goes_bands = xr.open_dataset(a_goes)
for band in range(7,16):
    goes_bands[F'CMI_C{band:02d}']-= 273.15 # Convertir las temperaturas de brillo de Kelvin Celsius 
    goes_bands[F'CMI_C{band:02d}'].attrs['units'] = 'C' # Actualizar las unidades en el dataset
bandas = hv.Dataset(goes_bands)
goes_bands

In [ ]:
goes_bands.t.data

## Banda 13

Esta región del infrarrojo **poco sensible a la absorción del vapor de agua**, en comparación con otras regiones del espectro infrarrojo. Lo que permite identificar y clasificar nubes y estimar la temperatura de brillo del tope de las nubes, **detectar indicadores de clima extremo** y la intensidad de los huracanes. Esta banda es muy utilizada para la creación de muchos compuestos RGB.

## Análisis de bandas individuales

Empecemos por abrir nuevamente la banda 02 pero remuestreada a aproximadamente 2 km y observemosla junto a la banda 13

Prueba cambiar las bandas que estan dezplegadas y analizá que es lo que observas. Si quieres probar otros mapas de colores puedes encontrar una lista en la documentacion de [Holoviews](https://holoviews.org/user_guide/Colormaps.html).

Tambien puedes utilizar el argumento `clim` para limitar el rango de la escala de colores, todo lo que este encima o debajo del umbral se redondeara a los limites de la paleta seleccionada.

Intenta  cambiar el color de la banda 13 a alguna paleta divergente y juega con `clim`.

In [ ]:
imagen_C02 =  bandas.to(hv.Image, ['x', 'y'], 'CMI_C01')
imagen_C13 =  bandas.to(hv.Image, ['x', 'y'], 'CMI_C13')

hv.Layout(imagen_C02.opts(cmap='Greys_r', colorbar=True, width=800, height=600, clim=(0,1), tools=['hover']) + 
          imagen_C13.opts(cmap='Greys', colorbar=True, width=800, height=600, tools=['hover'])
         ).cols(2)

## Sobreponiendo falso color a una sola banda

En este apartado crearemos una serie de productos en los cuales aplicaremos falso color para resaltar rasgos en las escenas.
Para esta seccion utilizaremos las bandas 02 y 13 del GOES.

En una figura utilizaremos la banda 02 como mapa base para sobreponer las nubes frías detectables en la banda 13 y en otra utilizaremos la banda 13 tanto como base y falso color.

In [ ]:
composicion = goes_bands[['CMI_C02','CMI_C13']].copy()
composicion['CMI_C13b'] = goes_bands['CMI_C13'].copy()
composicion['CMI_C13b'].data[composicion['CMI_C13b'].data > -30] = np.nan
hv_composicion = hv.Dataset(composicion)


baseC02 = hv_composicion.to(hv.Image, ['x', 'y'], 'CMI_C02').opts(cmap='Greys_r', clim=(0,1))
baseC13 = hv_composicion.to(hv.Image, ['x', 'y'], 'CMI_C13').opts(cmap='Greys', clim=(-80, 40))
color = hv_composicion.to(hv.Image, ['x', 'y'], 'CMI_C13b').opts(cmap='rainbow_r', colorbar=True, tools=['hover'], alpha=0.5, clim=(-80, -30))

hv.Layout(
    (baseC02 * color).opts(width=800, height=600, tools=['hover']) + 
    (baseC13 * color.opts(alpha=1)).opts(width=800, height=600, tools=['hover'])
         )

```{figure} img/SMN_20230829_22UTC.png
---
width: 50%
name: SMN_230829.
:align: center
---
Imagen intepretada del SMN a las 22UTC. _[SMN CONAGUA.](https://smn.conagua.gob.mx/tools/DATA/Formularios/Pron%C3%B3stico%20Meteorol%C3%B3gico%20General/Pron%C3%B3stico%20Meteorol%C3%B3gico%20General_6664.pdf)_
```

:::{warning}
Antes de cerrar este notebook, es recomendable detener, borrar todas las salidas ya que el despliegue de esta imagen hace que este notebook se vuelva muy pesado.

    Kernel-->Restart Kernel and Clear Outputs of All Cells
:::